Before starting let's go back to out main directory. In USERNAME write the user path you are using...

In [ ]:
import os 
os.chdir('C:/Users/USERNAME/')

During this phase, we will do two things:

->Get familiar with the functions used for motion capture.

->Reorient the global coordinate system of the cameras to make it much easier to later construct the bounding boxes used to detect the component being worked on.

Before doing anything, record some videos (always under 1 minute) with the following characteristics:

->Hand movements should not be too fast.

->The videos should start with the operator slowly opening and closing both hands above the component.

However, there is still one important consideration: in order to simplify the later description of volumes (bounding boxes of components) in 3D space, the global coordinate system must at least be consistent and aligned with certain key points (belonging to the unit on the workbench) chosen arbitrarily by the user.

We will therefore choose to record videos in which the user, moving their right index finger (marked by keypoint number 8), points to 3 locations on the component:
-The origin of the new coordinate system (referred to as O from now on)
-A point along the new X-axis  (referred to as X from now on)
-A point along the new Y-axis (referred to as Y from now on)

Important note! The user must ensure that these points are chosen so that there is already an orthogonal geometry between the X and Y points (for example, if thinking about a motherboard, it would be advisable to place these two points on two opposite corners).
Important note 2! To avoid "non-perfectly aligned global coordinate systems" due to small deviations between the real 3D position of the keypoint and the triangulated position calculated by the motion capture functions, it is recommended to place the X and Y points as far as possible from the chosen origin, while still remaining on the component

Per iniziare, andiamo a creare all'interno della cartella EasyMocap/data la cartella videos, e carichiamo al suo interno i file di calibrazione e i video che si vogliono utilizzare per la motion capture atta al fine tuning del sistema di riferimento.
In quanto alle caratteristiche dei video, valgono le stesse considerazioni utilizzate in fase di calibrazione.
Dopo aver caricato i video carichiamo anche i file extri.yml e intri.yml precedentemente ottenuti durante la fase di calibrazione e contenuti in extri_data.

In [ ]:
import os 
import shutil

os.makedirs('EasyMocap/data/videos', exist_ok=True)
shutil.copy('EasyMocap/yamls/intri.yml' ,'EasyMocap/data')
shutil.copy('EasyMocap/yamls/extri.yml' ,'EasyMocap/data')

Nel caso in cui combinaste pasticci, c'è sempre la funzione cleanhandmocap.py che vi permette di ripulire la cartella che 
state utilizzando.
La prossima funzione, da utilizzare una volta sola (poi il file rimane modificato) permette di cambiare alcuni parametri a seconda che l'operatore nel video è destro o mancino. Si consiglia il suo utlizzo.

In [ ]:
!python EasyMocap/apps/hopeitworks/cleanhandmocap.py

In [ ]:
#YOU HAVE TO RUN IT IN THE TERMINAL, without the ! before python
#!python EasyMocap/apps/hopeitworks/choosehand.py

Ok. Ora passiamo all'estrazione delle immagini tramite il prossimo notebook...assicuratevi di essere sempre nella directory principale, tramite il comando posto in alto.

In [ ]:
##ESTRAE IMMAGINI DA VIDEO
import time
import os
# Inizio del timer
start_time = time.time()

# ESTRAZIONE IMMAGINI
!python EasyMocap/apps/preprocess/extract_image.py EasyMocap/data

# Fine del timer
end_time = time.time()
elapsed_time = end_time - start_time
# Salva il tempo in un file txt
with open('EasyMocap/data/times.txt', 'a') as f:
    f.write(f't extract_image, {elapsed_time:.2f}\n')

In [ ]:
Al calcolo dei keypoints...

In [ ]:
##ESTRAE KEYPOINTS
import time
import os
import json
import shutil
# Inizio del timer
start_time = time.time()

## MANO SINISTRA
!python EasyMocap/apps/preprocess/extract_keypoints.py EasyMocap/data --mode mp-handl
## RINOMINA ANNOTS IN ANNOTSL PER CONVENIENZA DEL SUCCESSIVO ANNOT (DELLA MANO DESTRA)
os.rename('EasyMocap/data/annots', 'EasyMocap/data/annotsl')
## MANO DESTRA
!python EasyMocap/apps/preprocess/extract_keypoints.py EasyMocap/data --mode mp-handr


##INIZIO JOIN DELLE DUE CARTELLE ANNOTS

# Define the path to your main folder
main_folder_path = 'EasyMocap/data/annotsl'

# Get a list of sub-folders within the main folder
subfolders = [f.path for f in os.scandir(main_folder_path)]

# Loop through each sub-folder
for subfolder_path in subfolders:

    # Get a list of JSON files within the sub-folder
    json_files = [f.path for f in os.scandir(subfolder_path)]

    # Loop through the JSON files in the sub-folder
    for json_file_path in json_files:

        # You can now read and process the JSON file
        with open(json_file_path, 'r') as json_file:
            data = json.load(json_file)
            json_file_path_dest = json_file_path.replace("annotsl", "annots")

            # Step 1: Read the existing data from the destination JSON file
            with open(json_file_path_dest, 'r') as json_file:
                existing_data = json.load(json_file)

            # Step 2: Modify or add new data to the existing data
            existing_data["annots"][0]["handl2d"] =  data["annots"][0]["handl2d"]
            existing_data["annots"][0]["bbox_handl2d"] =  data["annots"][0]["bbox_handl2d"]

            # Step 3: Write the updated data back to the JSON file
            with open(json_file_path_dest, 'w') as json_file:
                json.dump(existing_data, json_file, indent=4)

shutil.rmtree('EasyMocap/data/annotsl')
##FINE JOIN DELLE DUE CARTELLE ANNOTS

# Fine del timer
end_time = time.time()
elapsed_time = end_time - start_time
# Salva il tempo in un file txt
with open('EasyMocap/data/times.txt', 'a') as f:
    f.write(f't extract_keypoints, {elapsed_time:.2f}\n')

##in times.txt, SCRIVERE A INIZIO FILE NUMERO FRAME E NUMERO FPS CON QUESTO FORMATO EX. (#FRAMES, 434 /n #FPS, 9)

E infine alla triangolazione...

In [ ]:
##TRIANGOLAZIONE K2D->K3D
import time

# Inizio del timer
start_time = time.time()

!python EasyMocap/apps/fit/triangulate1p.py --cfg_data EasyMocap/config/recon/mv1p.yml --opt_data args.path EasyMocap/data args.out EasyMocap/data/output-keypoints3d --cfg_exp EasyMocap/config/recon/triangulator-mv1p-total.yml --opt_exp  args.debug True

# Fine del timer
end_time = time.time()
elapsed_time = end_time - start_time
# Salva il tempo in un file txt
with open('EasyMocap/data/times.txt', 'a') as f:
    f.write(f't triangulate, {elapsed_time:.2f}\n')

Se tutto è andato a buon fine, possiamo continuare andando nella cartella data->match (guardate usando le cartelle di windows è piu veloce scorrere le immagini con la freccia tenendo premuto, trovate le immagini in C:\Users\USERNAME\EasyMocap\data\output-keypoints3d\match) e scegliere i frame corrispondenti a indice in punto O, X e Y ovvero in cui il dito è meglio collocato nei 3 punti di cui parlavamo prima.
Ora non rimane che segnarsi i numeri corrispondenti a questi 3 frame, andare nella cartella outputkeypoints3d, aprire i corrispondenti file json dei frame scelti e scrivere i valori delle cordinate handr 8 (mano destra indice) in un file chiamato newcordinate.txt (salvandolo in data) con la seguente struttura (trovate un esempio in data, volendo modificate quello):

O: 0.1656968, 0.4882543, 0.2202636

X: 0.0809912, 0.4186054, 0.2154134

Y: 0.2596883, 0.3956782, 0.2286461

Ora iniziamo alla fase di riorientamento

*PRIMO STEP*

Trasformiamo il file di calibrazione estrinseco in modo che questo descriva le rototraslazioni necessarie per passare da 
sistema di riferimento GLOBALE a sistema LOCALE.
Questo perchè la calibrazione da come output un file extri in cui invece è descritto l'inverso, ovvero la rototraslazione necessaria
per passare dal sistema di riferimento LOCALE (quindi quello di ogni telecamera) a quello GLOBALE (in questo caso fissato nell'origine della scacchiera usata durante la fase di calibrazione.

In [ ]:
!python EasyMocap/apps/hopeitworks/rototrasla_yaml.py EasyMocap/data/extri.yml

SECONDO STEP
Ora, con la prossima funzione possiamo plottare il file .yml ottenuto dalla precedente funzione.
I sistemi di riferimento locali assumono quindi un significato tridimensionale tangibile e ricollegabile 
alla configurazione reale del sistema...

In [ ]:
#!python EasyMocap/apps/hopeitworks/visualizza_yaml.py EasyMocap/data/extri_inv.yml

TERZO STEP 
Ora possiamo passare al effettivo riorientamento del sistema di riferimento globale utilizzando la seguente funzione

In [ ]:
!python EasyMocap/apps/hopeitworks/riorienta_sistemaglobale.py EasyMocap/data/extri_inv.yml EasyMocap/data/newcordinate.txt

QUARTO STEP 
Invocando di nuovo la funzione per la visualizzazione 3d possiamo fare un check se il sistema globale è stato
riorientato correttamente...

In [ ]:
#!python EasyMocap/apps/hopeitworks/visualizza_yaml.py EasyMocap/data/extri_inv_reoriented.yml

QUINTO STEP
Risulta essenziale tornare al pov originale utilizzato inizialmente. 
Questo viene fatto rototraslando di nuovo il file per tornare a una rappresentazione locale->globale.

In [ ]:
!python EasyMocap/apps/hopeitworks/rototrasla_yaml.py EasyMocap/data/extri_inv_reoriented.yml

Ora rendiamo il nuovo file extri quello "ufficiale" tramite la funzione sostituisciextri.py

In [ ]:
!python EasyMocap/apps/hopeitworks/sostituisciextri.py EasyMocap/data

Ora controlliamo se il nostro nuovo sistema di riferimento è corretto e allineato al nostro componente tramite la funzione cube utilizzata in fase di calibrazione..

In [ ]:
!python EasyMocap/apps/calibration/check_calib.py EasyMocap/data --out EasyMocap/data --mode cube --write

La maggior parte delle volte, vi saranno degli aggiustamenti da fare...niente paura, è molto semplice.
Il nostro sistema di riferimento potrebbe essere non centrato sull'origine, oppure gli assi x e y potrebbero non essere allineati al nostro componente.
Per risolvere baster ripetere i passaggi visti precedentemente in maniera iterativa, fino a raggiungere il risultato sperato...
Basterà cambiare (uno alla volta per non stravolgere troppo) i valori relativi a una singola variabile (o O o X o Y) all'interno del file newtuning.txt (ricordarsi che i valori sono in metri).
Inoltre ricordarsi che la funzione riorienta cerca di far passare gli assi del sistema globale "nuovo" per le posizioni specificate nell'asse globale "precedente" (per intenderci).
Una volta cambiato newtuning possiamo procedere con la cella sotto:

In [ ]:
#ITERAZIONE ripetiamo comandi precedenti e controlliamo proiezione cubo...
!python EasyMocap/apps/hopeitworks/rototrasla_yaml.py EasyMocap/data/extri.yml
!python EasyMocap/apps/hopeitworks/riorienta_sistemaglobale.py EasyMocap/data/extri_inv.yml EasyMocap/data/newtuning.txt
!python EasyMocap/apps/hopeitworks/rototrasla_yaml.py EasyMocap/data/extri_inv_reoriented.yml
!python EasyMocap/apps/hopeitworks/sostituisciextri.py EasyMocap/data
!python EasyMocap/apps/calibration/check_calib.py EasyMocap/data --out EasyMocap/data --mode cube --write

OPZIONE A: ulteriore tuning necessario-> procediamo con il ripristinare il nostro file newtuning, scrivere i nuovi valori voluti, e runnare di nuovo la cella #ITERAZIONE

In [ ]:
!python EasyMocap/apps/hopeitworks/ripristinatuning.py EasyMocap/data

OPZIONE B: siamo contenti dei risultati.
In questo caso possiamo salvare i risultati in yamls con la cella sotto. Il nostro compito è concluso :)

In [ ]:
import shutil

shutil.copy('EasyMocap/data/extri.yml', 'EasyMocap/yamls/REORIENTED')
shutil.copy('EasyMocap/data/intri.yml', 'EasyMocap/yamls/REORIENTED')

See you in the next tutorial